# Transfer learning (fine-tuning)

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

* flower dataset 으로 해 보자.

## Learning from scratch

In [1]:
import keras
from keras import backend as K
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Dense, Flatten, Dropout, Input
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [2]:
keras.__version__

'2.0.6'

In [3]:
np.random.seed(0)

In [4]:
def build_model_functional(input_shape, output_units):
    input_tensor = Input(input_shape)
    net = input_tensor
    n_filters = 32

    for _ in range(3):
        net = Conv2D(n_filters, [3,3], padding='same', use_bias=False)(net)
        net = BatchNormalization()(net)
        net = Activation('relu')(net)
        net = Conv2D(n_filters, [3,3], padding='same', use_bias=False)(net)
        net = BatchNormalization()(net)
        net = Activation('relu')(net)
        net = MaxPooling2D(padding='same')(net)
        net = Dropout(0.3)(net)
        
        n_filters *= 2

    net = Flatten()(net)
    net = Dense(output_units, activation='softmax')(net)

    model = Model(input_tensor, net)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
# build net
# without data augmentation
datagen = ImageDataGenerator(rescale=1./255)

In [13]:
batch_size = 128
train_generator = datagen.flow_from_directory(directory='./data/flower_photos/train/', 
                                              target_size=[64, 64], 
                                              batch_size=batch_size)
test_generator = datagen.flow_from_directory(directory='./data/flower_photos/test/',
                                             target_size=[64, 64],
                                             batch_size=batch_size)
model = build_model_functional([64, 64, 3], 5)

Found 3306 images belonging to 5 classes.
Found 364 images belonging to 5 classes.


In [14]:
result = model.fit_generator(generator=train_generator,
                             steps_per_epoch=3306//batch_size,
                             epochs=60,
                             validation_data=test_generator,
                             validation_steps=364//batch_size,
                             verbose=2)

Epoch 1/60
5s - loss: 2.1803 - acc: 0.4106 - val_loss: 1.6292 - val_acc: 0.2305
Epoch 2/60
4s - loss: 1.2412 - acc: 0.5505 - val_loss: 1.9736 - val_acc: 0.2383
Epoch 3/60
5s - loss: 1.1315 - acc: 0.5924 - val_loss: 2.3536 - val_acc: 0.2383
Epoch 4/60
4s - loss: 1.0586 - acc: 0.6243 - val_loss: 2.6283 - val_acc: 0.2383
Epoch 5/60
4s - loss: 0.9611 - acc: 0.6432 - val_loss: 2.6265 - val_acc: 0.2500
Epoch 6/60
4s - loss: 0.9264 - acc: 0.6592 - val_loss: 3.0631 - val_acc: 0.2578
Epoch 7/60
4s - loss: 0.8680 - acc: 0.6784 - val_loss: 3.3860 - val_acc: 0.2266
Epoch 8/60
4s - loss: 0.7874 - acc: 0.7192 - val_loss: 4.1670 - val_acc: 0.2422
Epoch 9/60
4s - loss: 0.7961 - acc: 0.7081 - val_loss: 3.3584 - val_acc: 0.2266
Epoch 10/60
5s - loss: 0.7152 - acc: 0.7389 - val_loss: 4.0701 - val_acc: 0.2500
Epoch 11/60
4s - loss: 0.7435 - acc: 0.7263 - val_loss: 3.7318 - val_acc: 0.2109
Epoch 12/60
4s - loss: 0.6426 - acc: 0.7662 - val_loss: 3.4272 - val_acc: 0.2812
Epoch 13/60
4s - loss: 0.6285 - acc: 

In [15]:
print "{:.2%}".format(np.average(result.history['val_acc'][-5:]))

69.06%
